In [ ]:
import numpy as np
import torchio as tio
import matplotlib.pyplot as plt

import codebase.codebase_settings as cbs

In [ ]:
def comparison_plot(images, nslice: int):
    nimages = len(images)
    fig, axes = plt.subplots(1, nimages, num=1, clear=True, figsize=(nimages * 4, nimages))
    for i in range(nimages):
        axes[i].imshow(images[i][0, :, :, nslice])
    plt.tight_layout()
    plt.show()

In [ ]:
raw_data_path = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data'
raw_data_path = cbs.DATA_PATH / 'hecktor2022'
id = 'CHUP-052'
id = 'CHUP-017'
id = 'CHUP-028'

In [ ]:
raw_ct = tio.ScalarImage(raw_data_path / 'images' / (id + '__CT.nii.gz'))
print(raw_ct.shape, raw_ct.spacing)
raw_pt = tio.ScalarImage(raw_data_path / 'images' / (id + '__PT.nii.gz'))
print(raw_pt.shape, raw_pt.spacing)
raw_lb = tio.LabelMap(raw_data_path / 'labels' / (id + '.nii.gz'))
print(raw_lb.shape, raw_lb.spacing)

In [ ]:
comparison_plot([raw_ct.data, raw_pt.data, raw_lb.data], nslice=10)

In [ ]:
data_path = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data' / 'processed_256x256' / 'train'
data_path = cbs.DATA_PATH / 'hecktor2022' / 'processed_256x256' / 'train'

In [ ]:
ct = tio.ScalarImage(data_path / 'images' / (id + '__CT.nii.gz'))
print(ct.shape, ct.spacing)
pt = tio.ScalarImage(data_path / 'images' / (id + '__PT.nii.gz'))
print(pt.shape, pt.spacing)
lb = tio.LabelMap(data_path / 'labels' / (id + '.nii.gz'))
print(lb.shape, lb.spacing)

In [ ]:
comparison_plot([ct.data, pt.data, lb.data], nslice=80)

In [ ]:
subvolume_data_path = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data' / 'processed_256x256' / 'subvolume_32' / 'train'
subvolume_data_path = cbs.DATA_PATH / 'hecktor2022' / 'processed_256x256' / 'subvolume_32' / 'train'

In [ ]:
sub_id = '_141'
sub_input = np.load(subvolume_data_path / 'images' / (id + sub_id + '__input.npy'))
sub_ct = sub_input[0:1, ...]
print(sub_ct.shape)
sub_pt = sub_input[1:, ...]
print(sub_pt.shape)
sub_lb = np.load(subvolume_data_path / 'labels' / (id + sub_id + '__label.npy'))
print(sub_lb.shape)

In [ ]:
comparison_plot([sub_ct, sub_pt, sub_lb], nslice=31)